In [ ]:
#import library
import os
import numpy as np
import scipy.io
import pandas as pd
from scipy import signal
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from mne.decoding import CSP
import mne
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

filePath = '/Users/yamamotokouhei/subjectsData/data20221122'
brainAdress = ['Fp1','AF7','AF3','F1','F3','F5','F7','FT7','FC5',
'FC3','FC1','C1','C3','C5','T7','TP7','CP5','CP3','CP1','P1','P3',
'P5','P7','P9','PO7','PO3','O1','Iz','Oz','POz','Pz','CPz','Fpz','Fp2',
'AF8','AF4','AFz','Fz','F2','F4','F6','F8','FT8','FC6','FC4','FC2',
'FCz','Cz','C2','C4','C6','T8','TP8','CP6','CP4','CP2','P2','P4',
'P6','P8','P10','PO8','PO4','O2']

# dataクラス
class O_Data:
    def __init__(self, eeg, index, label):
        self.eeg = eeg
        self.index = index
        self.label = label
        self.stack = []

    def push(self, item):
        self.stack.append(item)

o_data65_label1 = list()#label1のデータ
o_data65_label2 = list()#label2のデータ
o_data = list()
label_data = list()#labelを結合したデータ
task1_data = list()
task2_data = list()

All_data = list()  # 全データを結合

def Standardization(data): #標準化
    after_data = scipy.stats.zscore(data)
    #b = np.average(after_data)
    #c = np.var(after_data)
    return after_data

file_name = list()  # すべての.matファイルの名前
for file in os.listdir(filePath):
    base, ext = os.path.splitext(file)
    if ext == '.mat':
        file_name = sorted(file_name)
        file_name.append(file)
print('Filelist')
print(file_name)

ModuleNotFoundError: No module named 'mne'

In [ ]:
#label1の設定
file_name1 = [file_name[0],file_name[1],file_name[2]]
file_num1 = len(file_name1)
#label2の設定
file_name2 = [file_name[3],file_name[4],file_name[5]]
file_num2 = len(file_name2)
# 変数など
trial_num = 30
all_trial = len(file_name1)+len(file_name2) * trial_num
all_d = 0
CH = 64
#label1を読み込む
for s in range(file_num1):
    Dictionary = scipy.io.loadmat(filePath+"/"+file_name1[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:64, :]
                index = b[1]
                samplerate = b[2]
                label = b[5][0, 0]
                o_data = O_Data(eeg, index, label)
                o_data65_label1.append(o_data)
#label2を読み込む
for s in range(file_num2):
    Dictionary = scipy.io.loadmat(filePath+"/"+file_name2[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:64, :]
                index = b[1]
                samplerate = b[2]
                label = b[5][0, 0]
                o_data = O_Data(eeg, index, label)
                o_data65_label2.append(o_data)
print("data_loading")

: 

In [ ]:
c1 = o_data65_label1[0].eeg[: , 0:8200]
c2 = o_data65_label2[0].eeg[: , 0:8200]
for n in range(len(file_name1)*trial_num-1):  # all_trial
    g1 = o_data65_label1[n+1].eeg[: , 0:8200]
    g2 = o_data65_label2[n+1].eeg[: , 0:8200]
    c1 = np.dstack([c1, g1])
    c2 = np.dstack([c2, g2])
all_data = np.dstack([c1, c2])
all_data = all_data.transpose(2,0,1)
np.shape(all_data)
all_data = all_data.astype(np.double)


#ラベル生成
# label = np.zeros(180,dtype=int)
# label[90:] = 1
a = list()
b = list() 
for n in range(len(file_name1)*trial_num):  # all_trial
    label1 = o_data65_label1[n].label
    label2 = o_data65_label2[n].label
    a.append(label1)
    b.append(label2)
label = a+b
label = np.array(label)

: 

In [ ]:
from sklearn.decomposition import FastICA
ICA = FastICA(n_components=20,whiten="arbitrary-variance", max_iter = 200,tol = 1e-2,random_state=42)#20個の基底（コンポネント）を作る

#trialごとのICA
for s in range(len(label)):
    X_transformed = ICA.fit_transform(all_data[s])
    A_ =  ICA.mixing_.T #混合行列
    all_data[s] = np.dot(X_transformed[0], A_)

filter3 = signal.firwin(numtaps=51, cutoff=[5, 40], fs=1024, pass_zero=False)
for i in range(180):
    all_data[i] -=np.mean(all_data[i][:][:])
    for j in range(64):
        all_data[i][j] = signal.lfilter(filter3, 1, all_data[i][j][:])
        all_data[i][j] = Standardization(all_data[i][j][:])
print(np.shape(all_data))

: 

In [ ]:
a = 0
N = 8200            # サンプル数
dt = 1/1024          # サンプリング間隔
t = np.arange(0, N*dt, dt) # 時間軸
freq = np.linspace(0, 1.0/dt, N) # 周波数軸
g = list()
for s in range(len(label)):
    for k in range(64):
        F = np.fft.fft(all_data[s][k])
        Amp = np.abs(F)
        a += Amp[0:240]
    g.append(a/64)   
X_train, X_test, Y_train, Y_test = train_test_split(g, label, test_size=0.25,random_state=42)
model =SVC(kernel='linear')
model.fit(X_train,Y_train)
print(model.score(X_train, Y_train))
print(model.score(X_test, Y_test))

params = {
    "C":np.arange(0.1,1,0.05),
    "kernel":["linear", "poly", "rbf", "sigmoid"],
    "gamma":np.arange(0.0001,0.1,0.05)
}
grid = GridSearchCV(model, params,scoring="accuracy", cv=10)

grid.fit(X_train, Y_train)
print(grid.best_score_)
print(grid.best_params_)

pred = grid.predict(X_test)
print(classification_report(Y_test, pred))


: 